# DataFabrikken Hackathon
Welcome to the Hackathon! This Jupyter Python-notebook can help you getting started with the datasets provided.

We start by importing some common Python libraries that we might need, in the cell below. Execute the commands/content in the cells by using "Run" from the menu (with the cell that you want to run selected), or use "Ctrl+Enter" on the keyboard.

In [ ]:
import pandas as pd
import numpy as np
import re
from datetime import datetime
import math
import timeit
import requests
import io
import time
import matplotlib.pyplot as plt
import seaborn as sns

# Energy Datasets from municipalitites

The datasets are based on municipality source data.

There are four types of datasets that are connected:
- Location: Describes a location (usually a building) that we have energy data for.
- Measurement point: Describes a measurement point with a unique ID, what the measurement point is, and the unit of measurement.
- LocationMeasurementPoint: Connects a measurement point to a location and gives a local name for the measurement point. Some measurement points are used by several locations, for instance units for temperature measurement from public measurement stations from The Metheorological Institute (met.no)
- Measurements: Timestamped measurement value from a measurement point.

#### The descriptions of the individual datasets are shown below.

## Location
| Feltnavn | Beskrivelse  |
| -------- | ------------ |
| location | Navn på lokasjonen, f.eks. navn på skole eller barnehage |
| municipality_code | Kommunekoden for lokasjonen. Kodene er definert i datasettet [Standard for kommuneinndeling](https://www.ssb.no/klass/klassifikasjoner/131/koder) hos SSB. |
| location_type | Type lokasjon. Lovlige verdier: Bygning, Rom, Annet |
| building_type | Type bygning. Lovlige verdier: Skole, Barnehage, Helsebygg, Administrasjonsbygg, Idrettsbygg, Kirkelig bygg, Beredskapsbygg, Bydels- og fritidsbygg, Kulturbygg, Varmesentral, Annet bygg |
| address | Adresse til lokasjonen / bygget |
| latitude | Breddegrad for lokasjonen i desimalgrader |
| longitude | Lengdegrad for lokasjonen i desimalgrader |
 
## MeasurementPoint
| Feltnavn | Beskrivelse  |
| -------- | ------------ |
| meter_id | Unik id for målepunktet. Eksempler: EID_972418013_00290, Eklima_82290_TAM |
| meter_type | Beskriver hva slags type målepunkt det er. Eksempler er Fastkraft, Temperatur, Graddager, Varmepumpe |
| unit | Enhet som måleren måler, f.eks. kWh, °C |
| meter_level | Vanligvis en av følgende to verdier: Forbruksmåler, Annen måler |
 
## LocationMeasurementPoint
| Feltnavn | Beskrivelse  |
| -------- | ------------ |
| location | Navn på lokasjonen (se Location) |
| municipality_code | Kommunekoden for lokasjonen |
| meter_id | Unik id for målepunktet (se MeasurementPoint) |
| meter_name | Lokalt navn for målepunktet (unikt bare innenfor lokasjonen). Eksempler: Temp middel, Fastkraft, Fastkraft 67756 |
 
## Measurements
| Feltnavn | Beskrivelse  |
| -------- | ------------ |
| timestamp | Tidsstempel med dato og klokkeslett, f.eks. 2010-01-04 06:00:00 |
| meter_id | Unik id for målepunkt (se MeasurementPoint) |
| value | Måleravlesning. Vanligvis flyttall med komma som separator, f.eks. 4,20 |

### You can work with the datasets directly in this example notebook (or create your own), and/or download them to your local PC by going to hotell.difi.no.

### The following datasets are available for download:

Bodø
https://hotell.difi.no/?dataset=bodo/energy/location-measurement-points
https://hotell.difi.no/?dataset=bodo/energy/locations
https://hotell.difi.no/?dataset=bodo/energy/measurement-points
https://hotell.difi.no/?dataset=bodo/energy/measurements

Drammen
https://hotell.difi.no/?dataset=drammen/energy/location-measurement-points
https://hotell.difi.no/?dataset=drammen/energy/locations
https://hotell.difi.no/?dataset=drammen/energy/measurement-points
https://hotell.difi.no/?dataset=drammen/energy/measurements

Stavanger
https://hotell.difi.no/?dataset=stavanger/energy/location-measurement-points
https://hotell.difi.no/?dataset=stavanger/energy/locations
https://hotell.difi.no/?dataset=stavanger/energy/measurement-points
https://hotell.difi.no/?dataset=stavanger/energy/measurements

Trondheim
https://hotell.difi.no/?dataset=trondheim/energy/location-measurement-points
https://hotell.difi.no/?dataset=trondheim/energy/locations
https://hotell.difi.no/?dataset=trondheim/energy/measurement-points
https://hotell.difi.no/?dataset=trondheim/energy/measurements

### Example: Get the datasets from Bodø municipality and load into Python Pandas dataframes
Here are some examples of getting the datasets for Bodø municipality and looking at the dataset sizes and data contained in the datasets.

In [ ]:
# Get the Bodø locations and measurement points

url = 'https://hotell.difi.no/download/bodo/energy/location-measurement-points'

df_location_measurement_points_bodo = pd.read_csv(url, sep=';')
# Display the size of the dataset
print(df_location_measurement_points_bodo.shape)
# Display some of the data contained in the dataset
df_location_measurement_points_bodo.head()

In [ ]:
# Get the Bodø locations

url = 'https://hotell.difi.no/download/bodo/energy/locations'

df_locations_bodo = pd.read_csv(url, sep=';')
print(df_locations_bodo.shape)
df_locations_bodo.head()

In [ ]:
# Show how a column's values is distributed in the data set
print(df_locations_bodo['building_type'].value_counts())

# Plot the same data in a horizontal bar plot
df_locations_bodo['building_type'].value_counts().plot(kind='barh').set_title('Count of building types in Bodø dataset')
plt.show()

In [ ]:
# Get the Bodø measurement points

url = 'https://hotell.difi.no/download/bodo/energy/measurement-points'

df_measurement_points_bodo = pd.read_csv(url, sep=';')
print(df_measurement_points_bodo.shape)
df_measurement_points_bodo.head()

In [ ]:
# Get the Bodø measurements

url = 'https://hotell.difi.no/download/bodo/energy/measurements'

df_measurements_bodo = pd.read_csv(url, sep=';', low_memory=False)
print(df_measurements_bodo.shape)
df_measurements_bodo.head()

In [ ]:
# Example of joining/merging two datasets on a column that exists in both datasets

df_merged = df_locations_bodo.merge(df_location_measurement_points_bodo, on='location', how='inner')
df_merged.head(n=3)

These are just some very  brief and simple examples of how to work with Pandas dataframes.

Use the top menu or keyboard shortcuts to add, delete and move between cells to write code.

### Good luck with the Hackathon!